In [25]:
import pandas as pd
import numpy as np
import re

import torch
import torch.nn as nn
import torch.optim as optim

#from torchtext.datasets import Multi30k
#from torchtext.data import Field, BucketIterator

#import spacy
import numpy as np

import random
import math
import time

In [26]:
tmp = pd.read_csv('simpsons_script_lines.csv', sep=',', error_bad_lines=False)#, nrows=52608)

b'Skipping line 8084: expected 13 fields, saw 20\nSkipping line 52607: expected 13 fields, saw 21\nSkipping line 59910: expected 13 fields, saw 21\n'
b'Skipping line 71801: expected 13 fields, saw 20\nSkipping line 73539: expected 13 fields, saw 21\nSkipping line 77230: expected 13 fields, saw 21\nSkipping line 78953: expected 13 fields, saw 21\nSkipping line 81138: expected 13 fields, saw 20\nSkipping line 86746: expected 13 fields, saw 22\nSkipping line 101154: expected 13 fields, saw 21\nSkipping line 115438: expected 13 fields, saw 20\nSkipping line 117573: expected 13 fields, saw 22\nSkipping line 130610: expected 13 fields, saw 22\n'
b'Skipping line 152970: expected 13 fields, saw 22\nSkipping line 153017: expected 13 fields, saw 20\nSkipping line 153018: expected 13 fields, saw 30\nSkipping line 154080: expected 13 fields, saw 20\nSkipping line 154082: expected 13 fields, saw 20\nSkipping line 154084: expected 13 fields, saw 20\nSkipping line 154086: expected 13 fields, saw 20\n

In [27]:
character = 'Marge Simpson'
#character = 'Homer Simpson'
#character = 'Bart Simpson'
#выбираем персонажа

In [28]:
array_of_answers = []
for i, t in enumerate(tmp.episode_id[:-1]):
    try:
        if tmp.episode_id[i]==tmp.episode_id[i+1] and tmp.number[i]+1 == tmp.number[i+1] and tmp.raw_character_text[i+1] !=tmp.raw_character_text[i] and tmp.speaking_line[i]==True and tmp.speaking_line[i+1]==True and tmp.raw_location_text[i+1] == tmp.raw_location_text[i] and (tmp.raw_character_text[i+1] == character or tmp.raw_character_text[i] == character):
            if tmp.raw_text[i].find('(TO ') != -1:
                to_start = tmp.raw_text[i].find('(TO ')
                to_end = tmp.raw_text[i][to_start:].find(')')+to_start
                check_person =tmp.raw_text[i][to_start+4:to_end]
                check_person = check_person.lower().split(' and ')
                if any(s in tmp.raw_character_text[i+1].lower() for s in check_person):
                    writed_start = tmp.raw_text[i][to_end+1:]
                else:
                    continue
            else:
                writed_start = tmp.raw_text[i].split(str(tmp.raw_character_text[i])+': ')[1]
            if tmp.raw_text[i+1].find('(TO ') != -1:
                to_start = tmp.raw_text[i+1].find('(TO ')
                writed_end = tmp.raw_text[i+1][:to_start].split(str(tmp.raw_character_text[i+1])+': ')[1]
            else:
                writed_end = tmp.raw_text[i+1].split(str(tmp.raw_character_text[i+1])+': ')[1]
            array_of_answers.append([re.sub(r'\(.*\)', '', writed_start), re.sub(r'\(.*\)', '', writed_end), tmp.episode_id[i], tmp.raw_location_text[i], tmp.raw_character_text[i], tmp.raw_character_text[i+1]])           
    except:
        lalalala=2

In [29]:
array_of_answers

[['Nothing.',
  'Lisa, tell your father.',
  32,
  'Simpson Home',
  'Lisa Simpson',
  'Marge Simpson'],
 ['Lisa, tell your father.',
  'Mr. Bergstrom left today.',
  32,
  'Simpson Home',
  'Marge Simpson',
  'Lisa Simpson'],
 [' Did you hear that, Marge? She called me a baboon! The stupidest, ugliest, smelliest ape of them all!',
  "Homer, you are not allowed to have hurt feelings right now. There's a little girl upstairs who needs you. Her confidence in her father is shaken, and no little girl can be happy unless she has faith in her Daddy.",
  32,
  'Simpson Home',
  'Homer Simpson',
  'Marge Simpson'],
 ["Homer, you are not allowed to have hurt feelings right now. There's a little girl upstairs who needs you. Her confidence in her father is shaken, and no little girl can be happy unless she has faith in her Daddy.",
  ' Go Away.',
  32,
  'Simpson Home',
  'Marge Simpson',
  'Homer Simpson'],
 ['That was our wedding.',
  'Oh.',
  33,
  'Simpson Home',
  'Marge Simpson',
  'Homer S

In [30]:
array_to_df = []
for a in array_of_answers:
    if len(array_to_df) != 0 and array_to_df[-1] != str(a[2])+'\t'+a[3]+'\t'+a[4]+'\t'+a[0]:
        array_to_df.append(str(a[2])+'\t'+a[3]+'\t'+a[4]+'\t'+a[0])
    if len(array_to_df) == 0:
        array_to_df.append(str(a[2])+'\t'+a[3]+'\t'+a[4]+'\t'+a[0])
    array_to_df.append(str(a[2])+'\t'+a[3]+'\t'+a[5]+'\t'+a[1])

In [31]:
#Добавление дополнительных диалоговых линий, структура файла: 
#строки "номер_серии(например -5)\tЛокация(хоть что, главное чтобы было одинаковым)\tГерой(в случае вопроса пишем character, в случае ответа пишем имя перснажа)\tСтрочка_диалога"
with open('Additional_dialog_lines_' + character+'.txt','r',encoding='utf-8') as f:
    for line in f.readlines():
        array_to_df.append(line.strip())

In [33]:
array_to_df_2 = [a.split('\t') for a in array_to_df]

In [34]:
dataframe = pd.DataFrame(array_to_df_2)

In [35]:
dataframe

,0,1,2,3
0,32,Simpson Home,Lisa Simpson,Nothing.
1,32,Simpson Home,Marge Simpson,"Lisa, tell your father."
2,32,Simpson Home,Lisa Simpson,Mr. Bergstrom left today.
3,32,Simpson Home,Homer Simpson,"Did you hear that, Marge? She called me a bab..."
4,32,Simpson Home,Marge Simpson,"Homer, you are not allowed to have hurt feelin..."
5,32,Simpson Home,Homer Simpson,Go Away.
6,33,Simpson Home,Marge Simpson,That was our wedding.
7,33,Simpson Home,Homer Simpson,Oh.
8,33,Simpson Home,Marge Simpson,"Children, it's time for you to go to bed."
9,32,Simpson Home,Homer Simpson,"Sleep well, Maggie. Ah, three for three."


In [36]:
dataframe.to_csv(character+'_dialogs.csv')

In [37]:
len(array_to_df_2)

22408

In [38]:
context_ = 1
#выбираем длину контекста

In [39]:
array_of_responses = []
for i in range(len(array_to_df_2)-context_):
    tmp_ = [array_to_df_2[i+j_][0] for j_ in range(context_+1)]
    tmp__ = [array_to_df_2[i+j_][1] for j_ in range(context_+1)]
    #tmp_ = [array_to_df_2[i][0], array_to_df_2[i+1][0], array_to_df_2[i+2][0], array_to_df_2[i+3][0]]
    #tmp__ = [array_to_df_2[i][1], array_to_df_2[i+1][1], array_to_df_2[i+2][1], array_to_df_2[i+3][1]]
    if array_to_df_2[i+context_][2]==character and len(np.unique(tmp_))==1 and len(np.unique(tmp__))==1:
        array_of_responses.append([array_to_df_2[i+j_][3] for j_ in range(context_+1)][::-1])
        #array_of_responses.append([array_to_df_2[i+3][3], array_to_df_2[i+2][3], array_to_df_2[i+1][3], array_to_df_2[i][3]])

In [40]:
len(array_of_responses)

8682

In [41]:
array_of_responses

[['Lisa, tell your father.', 'Nothing.'],
 ["Homer, you are not allowed to have hurt feelings right now. There's a little girl upstairs who needs you. Her confidence in her father is shaken, and no little girl can be happy unless she has faith in her Daddy.",
  ' Did you hear that, Marge? She called me a baboon! The stupidest, ugliest, smelliest ape of them all!'],
 [" Children, it's time for you to go to bed.", 'Oh.'],
 ['Homie, did you straighten everything out...?',
  'Sleep well, Maggie. Ah, three for three.'],
 ['Homer! You promised!', 'Mmm... hors doovers.'],
 ["Homer, please. We've never thrown a party. Now we're gonna pay back all the friends who've invited us to their homes.",
  "I promised I wouldn't eat? Never! You lie!"],
 [" Oh! They're here! How does everything look?",
  'Hmmm.  Oh, baby! Mmmm. Yeah.'],
 ['Do we have enough glasses?', 'How do I look?'],
 ['Homer! Homer! Put a record on!', 'Do we have enough gag ice cubs?'],
 ['Children! Go!', 'What are all our friends nam

In [42]:
dataframe_responses = pd.DataFrame(array_of_responses)

In [43]:
dataframe_responses.shape

(8682, 2)

In [44]:
context_dict = {0:'response', 1:'context'}#, 2:'context/0', 3:'context/1'}#, 4:'context/2'}#, 5:'context/3'}

In [45]:
dataframe_responses = dataframe_responses.rename(columns = context_dict)

In [46]:
dataframe_responses

,response,context
0,"Lisa, tell your father.",Nothing.
1,"Homer, you are not allowed to have hurt feelin...","Did you hear that, Marge? She called me a bab..."
2,"Children, it's time for you to go to bed.",Oh.
3,"Homie, did you straighten everything out...?","Sleep well, Maggie. Ah, three for three."
4,Homer! You promised!,Mmm... hors doovers.
5,"Homer, please. We've never thrown a party. Now...",I promised I wouldn't eat? Never! You lie!
6,Oh! They're here! How does everything look?,"Hmmm. Oh, baby! Mmmm. Yeah."
7,Do we have enough glasses?,How do I look?
8,Homer! Homer! Put a record on!,Do we have enough gag ice cubs?
9,Children! Go!,What are all our friends names again?


In [47]:
dataframe_responses.to_csv(character+'_responses.csv', index=False)